# Smart Queue Monitoring System - Manufacturing Scenario

In this project, you will build a people counter app to reduce congestion in queuing systems by guiding people to the least congested queue. You will have to use Intel's OpenVINO API and the person detection model from their open model zoo to build this project. It demonstrates how to create a smart video IoT solution using Intel® hardware and software tools. This solution detects people in a designated area, providing the number of people in the frame.

## Overview of how it works
Your code should read the equivalent of command line arguments and loads a network and image from the video input to the Inference Engine (IE) plugin. A job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute Stick 2 and Intel® Arria® 10 FPGA.
After the inference is completed, the output videos are appropriately stored in the /results/[device] directory, which can then be viewed within the Jupyter Notebook instance.

## Demonstration objectives
* Video as input is supported using **OpenCV**
* Inference performed on edge hardware (rather than on the development node hosting this Jupyter notebook)
* **OpenCV** provides the bounding boxes, labels and other information
* Visualization of the resulting bounding boxes


## Step 0: Set Up

### 0.1: Import dependencies

Run the below cell to import Python dependencies needed for displaying the results in this notebook
(tip: select the cell and use **Ctrl+enter** to run the cell)

In [1]:
#Import your dependencies here
from demoTools.demoutils import *
import matplotlib.pyplot as plt

In [ ]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))

In [ ]:
pwd

### 0.2  (Optional-step): Original video without inference

If you are curious to see the input video, run the following cell to view the original video stream used for inference and people counter.

In [ ]:
videoHTML('People Counter Video1', ['./resources/manufacturing.mp4'])

## Step 1: Using Intel® Distribution of OpenVINO™ toolkit

We will be using Intel® Distribution of OpenVINO™ toolkit Inference Engine (IE) to locate people in frame.
There are five steps involved in this task:

1. Download the model using the open_model_zoo
2. Choose a device and create IEPlugin for the device
3. Read the Model using IENetwork
4. Load the IENetwork into the Plugin
5. Run inference.

### 1.1 Downloading Model

Write a command to download the  **person-detection-retail-0013** model in an IR format

In [ ]:
## Write your command here
!python /opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name person-detection-retail-0013 -o /intel

In [6]:
%%writefile person_detect.py

import numpy as np
from openvino.inference_engine import IENetwork,IECore
from openvino.inference_engine import IEPlugin
import os
import cv2
import argparse
import time
import matplotlib.pyplot as plt

class Queue:
    '''
    Class for dealing with queues
    '''
    def __init__(self):
        self.queues=[]

    def add_queue(self, points):
        self.queues.append(points)

    def get_queues(self, image):
        for q in self.queues:
            x_min, y_min, x_max, y_max=q
            frame=image[y_min:y_max, x_min:x_max]
            yield frame
    
    def check_coords(self, coords):
        d={k+1:0 for k in range(len(self.queues))}
        for coord in coords:
            for i, q in enumerate(self.queues):
                if coord[0]>q[0] and coord[2]<q[2]:
                    d[i+1]+=1
        return d

class PersonDetect:
    def __init__(self):
        self.plugin = None
        self.network = None
        self.mod_bin = None
        self.mod_xml = None
        self.exec_network = None
        self.input_blob = None
        self.output_blob = None
        self.net_input_shape = None
        self.frame = None


    #Loading the model
    def load_model(self,model,device):
        self.mod_bin = model+'.bin'
        self.mod_xml = model+'.xml'
        self.plugin = IECore()
        self.network = IENetwork(model=self.mod_xml, weights=self.mod_bin)
        self.exec_network = self.plugin.load_network(self.network,device_name=device)
        #raise NotImplementedError
        
        ##Get the supported layers of the network
        supp_layers = self.plugin.query_network(network=self.network, device_name = device)

        ###Check for any unsupported layers, and let the user
        unsupp = [l for l in self.network.layers.keys() if l not in supp_layers]
        if len(unsupp) != 0:
            print('unsupported layers found: {}'.format(unsupp))
            print('Check if the extensions are available in IECore')
            exit(1)

    def inp_out_blob(self):
        self.input_blob = next(iter(self.network.inputs))
        self.output_blob = next(iter(self.network.outputs))
        self.net_input_shape=self.network.inputs[self.input_blob].shape
        return(self.net_input_shape)

    def preprocess_input(self, image):
        self.x=self.net_input_shape[3]
        self.y=self.net_input_shape[2]
        self.Y,self.X,_ = np.shape(image)
        self.frame_resized = cv2.resize(image, (self.net_input_shape[3], self.net_input_shape[2]))
        self.frame = self.frame_resized.transpose((2,0,1))
        self.frame = self.frame.reshape(1, *self.frame.shape)
        return(self.frame,self.frame_resized)

        
    def predict(self, image):
        self.exec_network.start_async(request_id=0, inputs={self.input_blob: image})
        while True:
            status = self.exec_network.requests[0].wait(-1)               
            if status == 0:
                break
            else:
                time.sleep(1)
        return(self)

 
    def wait(self):
        status = self.exec_network.requests[0].wait(-1)
        return status

    def get_output(self):
        return self.exec_network.requests[0].outputs[self.output_blob]

    def preprocess_output(self,result,thresh):
        for i in range(len(result[0][0])):
            if result[0][0][i][2]>thresh:
                #print('entered')
                x1,y1,x2,y2 = result[0][0][i][3],result[0][0][i][4],result[0][0][i][5],result[0][0][i][6]
                
                self.x1 = x1*self.net_input_shape[3]*(self.X/self.x)
                self.y1 = y1*self.net_input_shape[2]*(self.Y/self.y)
                self.x2 = x2*self.net_input_shape[3]*(self.X/self.x)
                self.y2 = y2*self.net_input_shape[2]*(self.Y/self.y)
                
                yield(int(self.x1),int(self.y1),int(self.x2),int(self.y2))
            else:
                yield(0,0,0,0)


def main(args):
#extensions=args.extensions
    model= args.model
    device=args.device
    #visualise=args.visualise
    queue_param = args.queue_param
    max_people = int(args.max_people)
    #visualize = True
    video_file=args.video
    thresh = float(args.threshold)

    start=time.time()
    pd=PersonDetect()
    pd.load_model(model,device)
    mod_inp_shape = pd.inp_out_blob()
    print(f"Time taken to load the model is: {time.time()-start}")

    queue=Queue()
    #Queue Parameters
    if queue_param == 'retail':
       # For retail
        queue.add_queue([620, 1, 915, 562])
        queue.add_queue([1000, 1, 1264, 461])

    if queue_param == 'manufacturing':
       # For manufacturing
        queue.add_queue([15, 180, 730, 780])
        queue.add_queue([921, 144, 1424, 704])

    if queue_param == 'transportation':
       # For transportation
        queue.add_queue([150, 0, 1150, 794])
        queue.add_queue([1151, 0, 1915, 841])


    cap=cv2.VideoCapture(video_file)
    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    vid_write = cv2.VideoWriter(str(queue_param)+'.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10,(int(width),int(height)))
    #print(video_fps)
    #print(width,height)
    count = 0

    try:
        while cap.isOpened():
            ret, frame_org=cap.read()

            count+=1
            if not ret:
                break
            w,h,_ = np.shape(frame_org)
            #print(w,h)
            #for crop_frame in queue.get_queues(frame):

            frame, frame_resized = pd.preprocess_input(frame_org)
            pd.predict(frame)
            cords=[]
            # Get the output of inference
            if pd.wait() == 0:

                result = pd.get_output()
                #print(np.shape(result))

            for x1,y1,x2,y2 in pd.preprocess_output(result,thresh):
                cv2.rectangle(frame_org,(x1,y1),(x2,y2),(0,0,255),2)

                cords.append([x1,y1,x2,y2])
                #ppl=pd.count_person(result)
                #print('Number of people in Queue_'+str(count2)+' is {}'.format(ppl))
            #print(np.shape(frame_org))
            d = queue.check_coords(cords)
            stn_1 = 'No of person in Queue one :'
            stn_2 = 'No of person in Queue two :'
            cv2.putText(frame_org,stn_1+str(d[1]),(5,100),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3,cv2.LINE_AA)
            cv2.putText(frame_org,stn_2+str(d[2]),(5,135),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3,cv2.LINE_AA)

            mx_q1 = 'Too many persons in Queue one, move to other queue'
            mx_q2 = 'Too many persons in Queue two, move to other queue'    
            if d[1] > max_people:
                cv2.putText(frame_org,mx_q1,(5,700),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2,cv2.LINE_AA)
            if d[2] > max_people:
                cv2.putText(frame_org,mx_q2,(5,725),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2,cv2.LINE_AA)


            #videowriter = cv2.VideoWriter('Haar.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (500,550))

            if args.visualize:
                cv2.imshow('output',frame_org)
                vid_write.write(frame_org)
                cv2.waitKey(1)


            else:
                print(stn_1+str(d[1]))
                print(stn_2+str(d[2]))

        print(f"Time taken for model inference is: {time.time()-start}")
        print('model runs at {} FPS'.format(round(count/(time.time()-start),2)))


        cap.release()
        cv2.destroyAllWindows()
        vid_write.release()
    
    except Exception as e:
        print("Could not run Inference now", e)




if __name__=='__main__':
    parser=argparse.ArgumentParser()
    parser.add_argument('--model', required=True)
    parser.add_argument('--device', default='CPU')
    parser.add_argument('--extensions', default=None)
    
    parser.add_argument('--visualize', action='store_true')
    parser.add_argument('--video', default=None)
    parser.add_argument('--queue_param', default=None)
    parser.add_argument('--max_people', default=None)
    parser.add_argument('--threshold', default=None)
    
    args=parser.parse_args()

    main(args)


Writing person_detect.py


In [7]:
!python person_detect.py --model ./intel/person-detection-retail-0013/FP32/person-detection-retail-0013 --device CPU --video ./resources/manufacturing.mp4 --queue_param manufacturing --max_people 1 --threshold 0.8 --visualize

Time taken to load the model is: 1.0173790454864502
Time taken for model inference is: 33.373799085617065
model runs at 8.3 FPS


## Step 2 : Inference on a video

By now you should have already completed the inference code in <a href="person_detect.py">person_detect.py</a>. If you haven't done so already, then you should do it now.

The Python code should take in command line arguments for video, model etc.

While the type of command line options is up to you, the command below is an example 

```
python3 main.py -m ${MODELPATH} \
                -i ${INPUT_FILE} \
                -o ${OUTPUT_FILE} \
                -d ${DEVICE} \
                -pt ${THRESHOLD}\

```

##### The description of the arguments used in the argument parser is the command line executable equivalent.
* -m location of the pre-trained IR model which has been pre-processed using the model optimizer. There is automated support built in this argument to support both FP32 and FP16 models targeting different hardware
* -i  location of the input video stream
* -o location where the output file with inference needs to be stored (results/[device])
* -d type of Hardware Acceleration (CPU, GPU, MYRIAD, HDDL or HETERO:FPGA,CPU)
* -pt probability threshold value for the person detection

### 2.1 Creating job file

To run inference on the video, we need more compute power.
We will run the workload on several edge compute nodes present in the IoT DevCloud. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is written in Bash, and will be executed directly on the edge compute node.
You will have to create the job file by running the cell below.

In [9]:
%%writefile person_detect_job.sh
# The writefile magic command can be used to create and save a file

MODEL=$1
DEVICE=$2
VIDEO=$3
QUEUE=$4
OUTPUT=$5
PEOPLE=$6

mkdir -p $5

if [ $DEVICE = "HETERO:FPGA,CPU" ]; then
    #Environment variables and compilation for edge compute nodes with FPGAs
    source /opt/intel/init_openvino.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/2019R4_PL1_FP16_MobileNet_Clamp.aocx
fi

python3 person_detect_manu.py  --model ${MODEL} \
                                --visualise \
                                --queue_param ${QUEUE} \
                                --device ${DEVICE} \
                                --video ${VIDEO}\
                                --output_path ${OUTPUT}\
                                --max_people ${PEOPLE} \

Writing person_detect_job.sh


### 2.2 Understand how jobs are submitted into the queue

Now that we have the job script, we can submit the jobs to edge compute nodes. In the IoT DevCloud, you can do this using the `qsub` command.
We can submit people_counter to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- `-l` : this option let us select the number and the type of nodes using `nodes={node_count}:{property}`. 
- `-F` : this option let us send arguments to the bash script. 
- `-N` : this option let us name the job so that it is easier to distinguish between them.

Example using `qsub` command:

`!qsub person_detect_job.sh -l nodes=1:tank-870:i5-6500te -d . -F "models/intel/PATH-TO-MODEL DEVICE resources/manufacturing.mp4 bin/queue_param/manufacturing.npy results/manufacturing/DEVICE MAX-PEOPLE" -N JOB-NAME`

You will need to change the following variables, `models/intel/PATH-TO-MODEL`, `DEVICE`, `results/manufacturing/DEVICE`, `MAX-PEOPLE`, and `JOB-NAME` to the appropriate values.

If you are curious to see the available types of nodes on the IoT DevCloud, run the following optional cell.

In [10]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

/bin/sh: 1: pbsnodes: not found


Here, the properties describe the node, and number on the left is the number of available nodes of that architecture.

### 2.3 Job queue submission

Each of the cells below should submit a job to different edge compute nodes.
The output of the cell is the `JobID` of your job, which you can use to track progress of a job.

**Note** You can submit all jobs at once or one at a time. 

After submission, they will go into a queue and run as soon as the requested compute resources become available. 
(tip: **shift+enter** will run the cell and automatically move you to the next cell. So you can hit **shift+enter** multiple times to quickly run multiple cells)

If your job successfully runs and completes, it will output a video, `output_video.mp4`, and a text file, `stats.txt`, in the `results/retail/DEVICE` folder.

#### Submitting to an edge compute node with an Intel® CPU
In the cell below, write a script to submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core™ i5-6500TE processor</a>. The inference workload will run on the CPU.

In [ ]:
job_id = !/home/workspace/qsub person_detect_manu_job.sh -d . -l nodes=1:tank-870:i5-6500te -F "./intel/person-detection-retail-0013/FP32/person-detection-retail-0013.bin ./resources/retail.mp4" -N store_core 
print(job_id[0])

# FP32 used in CPU and F16 is used in VPU, FPGA,GPU 

#### Submitting to an edge compute node with Intel® Core CPU and using the onboard Intel® GPU
In the cell below, write a script to submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [ ]:
job_id = !qsub person_detect_manu_job.sh -d . -l nodes=1:tank-870:i5-6500te:intel-hd-530 -F "GPU ./intel/person-detection-retail-0013/FP32/person-detection-retail-0013.bin ./resources/retail.mp4" -N store_core 
print(job_id[0])

#### Submitting to an edge compute node with Intel® NCS 2 (Neural Compute Stick 2)
In the cell below, write a script to submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
job_id = !/home/workspace/qsub person_detect_manu_job.sh -d . -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "MYRIAD ./intel/person-detection-retail-0013/FP16/person-detection-retail-0013.bin ./resources/retail.mp4" -N store_core 
print(job_id[0])

#### Submitting to an edge compute node with IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)
In the cell below, write a script to submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core™ i5-6500te CPU</a> . The inference workload will run on the <a href="https://www.ieiworld.com/mustang-f100/en/"> IEI Mustang-F100-A10 </a> card installed in this node.

In [ ]:
job_id = !/home/workspace/qsub person_detect_manu_job.sh -d . -l nodes=1:tank-870:i5-6500te:iei-mustang-f100-a10 -F "HETERO:FPGA,CPU ./intel/person-detection-retail-0013/FP16/person-detection-retail-0013.bin ./resources/retail.mp4" -N store_core 
print(job_id[0])

### 2.4 Check if the jobs are done

To check on the jobs that were submitted, use a command to check the status of the job.

Column `S` shows the state of your running jobs.

For example:
- If `JOB ID`is in Q state, it is in the queue waiting for available resources.
- If `JOB ID` is in R state, it is running.

In [ ]:
# Enter your command here to check the status of your jobs
import liveQStat
liveQStat.liveQstat()

## Step 4: Assess Performance

This is where you need to write code to asses how well your model is performing. You will use the `stats.txt` file located in your results directory.
You need to compare the following timings for all the models across all 4 devices:

- Model loading time
- Average Inference Time
- FPS

Show your results in the form of a bar chart using matplotlib

In [ ]:
#TODO Write your code here for model loading time on all 4 device types

In [ ]:
#TODO Write your code here for model average inference time on all 4 device types

In [ ]:
#TODO Write your code here for model FPS on all 4 device types
!pip freeze